In [41]:
using DataFrames
using CSV
using Plots
using Polynomials
using LaTeXStrings

function analytical(x, y)
    return sin(3*pi*x)*sin(4*pi*y)
end

function get_tau(n, L, M, N)
    return 2/((4*(L^2+M^2) + 2*pi^2) + (4*(L^2+M^2)-2*pi^2)*cos(pi*(2*n-1)/(2*N)))
end

function get_n(N, num)
    if N == 1
        push!(num, 1)
        return 1
    end
    
    if N%2 == 0
        push!(num, N)
        get_n(Integer(round(N/2, digits = 0)), num)
    end
    
    if N%2 != 0
        push!(num, N)
        get_n(N-1, num)
    end
end



L = 5
M = 5

#for i = 1:6
#L = L*2
#M = M*2

hx = 1/(L)
hy = 1/(M)
error0 = 10^(-4)
xx = Vector(LinRange(0, 1, L+1))
yy = Vector(LinRange(0, 1, M+1))


max = 4*(L^2+M^2)
min = 2*pi^2
N = Integer(cld(log(2/error0),log((sqrt(max)+sqrt(min))/(sqrt(max)-sqrt(min)))))


u = zeros(Float64, (L+1, M+1, N+1))
anu = zeros(Float64, (L+1, M+1))
num = []
get_n(N, num)

num = reverse(num, dims = 1)
thetta = [1]
i = 1
while i <= (length(num) - 2)
    thetta_new = zeros(Float64, num[i+1])
    if num[i+2] == 2*num[i+1]
        for j = 1:num[i]
            thetta_new[2*j-1] = thetta[j]
            thetta_new[2*j] = 4*num[i] - thetta_new[2*j-1]
        end
    end
    
    if (i+2 == length(num)) & (num[i+2]%2 == 0)
        thetta = thetta_new
        thetta_new = zeros(Float64, num[i+2])
        for j = 1:num[i+1]
            thetta_new[2*j - 1] = thetta[j]
            thetta_new[2*j] = 4*num[i+1] - thetta_new[2*j-1] 
        end
    end
    
    
    if num[i+2] == num[i+1] + 1
        for j = 1:num[i]
            thetta_new[2*j-1] = thetta[j]
            thetta_new[2*j] = 4*num[i] - thetta_new[2*j-1] + 2
        end
        push!(thetta_new, num[i+2])
        i = i + 1
    end
    
    i = i + 1
    thetta = thetta_new
end

#print(thetta)
tau = get_tau.(thetta, L, M, N)
#print(tau)

for l = 2:L
    for m = 2:M
        x = hx*(l-1)
        y = hy*(m-1)
        
        anu[l, m] = sin(3*pi*x)*sin(4*pi*y)
    end
end
#print(anu)
for n = 1:N
    for l = 2:L
        for m = 2:M
            x = hx*(l-1)
            y = hy*(m-1)
            
            u[l, m, n+1] = u[l, m, n] + (tau[n]/hx^2)*(u[l+1, m, n] - 2*u[l, m, n] + u[l-1, m, n]) + 
            (tau[n]/hy^2)*(u[l, m+1, n] - 2*u[l, m, n] + u[l, m-1, n]) + tau[n]*25*pi^2*sin(3*pi*x)*sin(4*pi*y)
            
        end
    end
end

error = zeros(Float64, length(anu))
kx = Integer(L/5)
ky = Integer(M/5)
k = kx
numerical = u[:, :, N]
nn = zeros(Float64, (6, 6))
aa = zeros(Float64, (6, 6))
xxx = zeros(Float64, 6)
yyy = zeros(Float64, 6)
ee = zeros(Float64, (6, 6))
for i = 1:length(anu)
    error[i] = abs(numerical[i] - anu[i])
end
for i = 1:6
    for j = 1:6
        nn[i, j] = numerical[kx*(i-1)+1, ky*(j-1)+1]
        aa[i, j] = anu[kx*(i-1)+1, ky*(j-1)+1]
        ee[i, j] = abs(nn[i, j] - aa[i, j])
    end
    xxx[i] = xx[k*(i-1)+1]
    yyy[i] = yy[k*(i-1)+1]
end

norm = maximum(error)
df_num = DataFrame(nn, :auto)
rename!(df_num, [:"0.0", :"0.2", :"0.4", :"0.6", :"0.8", :"1.0"])
df_an = DataFrame(aa, :auto)
rename!(df_an, [:"0.0", :"0.2", :"0.4", :"0.6", :"0.8", :"1.0"])
df_ee = DataFrame(ee, :auto)
rename!(df_ee, [:"0.0", :"0.2", :"0.4", :"0.6", :"0.8", :"1.0"])

print("---------------------------------------------------------------------------------------------------------\n")
print("N = "*string(N)*", L = "*string(L)*", M = "*string(M)*"\n")
print("thetta(n) = "*string(thetta)*"\n")

print("Numerical solution:\n")
print(df_num)
print("\n")
print("Analytical solution:\n")
print(df_an)
print("\n")
print("Error:\n")
print(df_ee)
print("\n")
print("Max error = "*string(norm)*"\n")
print("---------------------------------------------------------------------------------------------------------\n")

#end
#print("\n")
#print(u[:, :, N])

---------------------------------------------------------------------------------------------------------
N = 16, L = 5, M = 5
thetta(n) = [1.0, 31.0, 15.0, 17.0, 7.0, 25.0, 9.0, 23.0, 3.0, 29.0, 13.0, 19.0, 5.0, 27.0, 11.0, 21.0]
Numerical solution:
6×6 DataFrame
 Row │ 0.0      0.2        0.4        0.6        0.8        1.0     
     │ Float64  Float64    Float64    Float64    Float64    Float64 
─────┼──────────────────────────────────────────────────────────────
   1 │     0.0   0.0        0.0        0.0        0.0           0.0
   2 │     0.0   0.884718  -1.4315     1.4315    -0.884718      0.0
   3 │     0.0  -0.546786   0.884718  -0.884718   0.546786      0.0
   4 │     0.0  -0.546786   0.884718  -0.884718   0.546786      0.0
   5 │     0.0   0.884718  -1.4315     1.4315    -0.884718      0.0
   6 │     0.0   0.0        0.0        0.0        0.0           0.0
Analytical solution:
6×6 DataFrame
 Row │ 0.0      0.2        0.4        0.6        0.8        1.0     
     │ Float64  